# Um guia de como testar as funções do anansi

## "Brincando" com os modelos nos bancos de dados

In [1]:
from anansi.tradingbot.models import *
from anansi.tradingbot import traders
from pony.orm import *

In [2]:
# Adicionando usuário
my_user_first_name = "Marcus"

create_user(first_name=my_user_first_name, 
                   last_name="Silva", 
                   email = "{}@email.com".format(my_user_first_name.lower()))

In [3]:
# Lendo todos os usuários
users = select(user for user in User)
users.show()

id|first_name|last_name|login_displayed_name|email           
--+----------+---------+--------------------+----------------
1 |Marcus    |Silva    |                    |marcus@email.com


In [4]:
# Criando uma operação padrão
my_user = User.get(first_name="Marcus")
create_operation(user=my_user)

## Instanciando um trader

In [5]:
my_op = Operation.get(id=1)
my_trader = traders.DefaultTrader(operation=my_op)

In [ ]:
type(my_trader.Classifier.parameters.time_frame)

In [6]:
my_trader.operation.position.quote_asset_amount

In [ ]:
import pendulum

In [ ]:
pendulum.from_timestamp(my_trader._now)

In [ ]:
my_user.operations.

## Klines tratadas e prontas para uso - com indicadores inclusive - utilizando o módulo "*klines*"

In [ ]:
from anansi.marketdata import klines

In [ ]:
BinanceKlines = klines.FromBroker(broker_name="binance", symbol="BTCUSDT", time_frame="1h")

In [ ]:
newest_klines = BinanceKlines.newest(2167)

### Aplicando indicador médias móveis

In [ ]:
newest_klines.apply_indicator.trend.simple_moving_average(number_of_candles=35, 
                                                          create_indicator_column=True)

In [ ]:
newest_klines

## Klines "cruas", utilizando diretamente o módulo "*data_brokers*"

In [ ]:
from anansi.marketdata import data_brokers

In [ ]:
BinanceBroker = data_brokers.BinanceDataWrapper() 

In [ ]:
my_klines = BinanceBroker.get_klines(symbol="BTCUSDT", time_frame="1m", show_only_desired_info=False)

In [ ]:
my_klines